# Jobs, Sessions, and Batch Execution (EXAM CRITICAL!)

> **Exam Weight**: Part of 15% (Section 4) | **Must Master**: ✅✅

## 🧠 Conceptual Overview

### Analogy: Restaurant Orders

| Concept | Restaurant Analogy |
|---------|-------------------|
| **Job** | A single order (one circuit) |
| **Batch** | Group order for a table (many independent circuits, served together) |
| **Session** | Reserved private room (dedicated access for sequential jobs) |

### When to Use Each

```
┌─────────────────────────────────────────────────────────────┐
│  Single Job: One-off experiments, quick tests              │
│  Batch: Parameter sweeps, independent circuits in parallel │
│  Session: VQE/QAOA (iterative), error correction           │
└─────────────────────────────────────────────────────────────┘
```

## Part 1: Job Lifecycle

In [ ]:
# Job submission pattern
print("Job Lifecycle:")
print("""
from qiskit_ibm_runtime import Sampler

sampler = Sampler(backend=backend)
job = sampler.run([circuit])  # Returns immediately (non-blocking)

# Job progresses through states:
# QUEUED → RUNNING → DONE

# Get results (blocks until done)
result = job.result()
""")

print("\nKey Job States:")
print("  - QUEUED: Waiting in queue")
print("  - RUNNING: Executing on hardware")
print("  - DONE: Completed successfully")
print("  - ERROR: Failed execution")
print("  - CANCELLED: User cancelled")

### job.result() vs job.status()

```python
# job.result() - BLOCKING
result = job.result()  # Waits until job completes
# Returns: PrimitiveResult object

# job.status() - NON-BLOCKING  
status = job.status()  # Returns immediately
# Returns: JobStatus enum (QUEUED, RUNNING, DONE, etc.)
```

**EXAM TIP**: `result()` blocks, `status()` does not!

## Part 2: Job Monitoring

In [ ]:
# Monitoring pattern
print("Job Monitoring:")
print("""
# Check status
print(job.status())  # Non-blocking

# Check queue position
position = job.queue_position()
print(f"Position: {position}")  # None if running/done

# Get job ID for later retrieval
job_id = job.job_id()
print(f"Job ID: {job_id}")

# Cancel if needed
job.cancel()
""")

print("\n✓ job_id() for tracking")
print("✓ queue_position() shows place in line")
print("✓ cancel() stops execution")

## Part 3: Job Retrieval

In [ ]:
# Retrieve job later
print("Job Retrieval:")
print("""
# Save job ID
job_id = job.job_id()

# Later: retrieve job
from qiskit_ibm_runtime import QiskitRuntimeService

service = QiskitRuntimeService()
retrieved_job = service.job(job_id)

# Get results from retrieved job
result = retrieved_job.result()
""")

print("\n✓ service.job(job_id) retrieves job")
print("✓ Works even after program closes")
print("✓ Job history persists in IBM Cloud")

## Part 4: Sessions (EXAM CRITICAL!)

### Why Sessions?

Sessions provide **reserved, sequential access** to quantum hardware:

```
Without Session:                  With Session:
Job1 → Queue → Run               Job1 → Reserved slot → Run
Job2 → Queue → Run               Job2 → Same slot → Run immediately
Job3 → Queue → Run               Job3 → Same slot → Run immediately
(Each job waits in queue!)        (Reserved access = no queue!)
```

### Session Benefits
- **Reduced queue time**: Reserved slot for multiple jobs
- **Iterative algorithms**: Perfect for VQE/QAOA where Job N depends on Job N-1
- **Cost-effective**: Pay for session time, not per-job overhead

In [ ]:
# Session pattern
print("Session Pattern (VQE/QAOA):")
print("""
from qiskit_ibm_runtime import Session, Sampler

# Create session with context manager
with Session(backend=backend) as session:
    # Pass session to primitive (NOT backend!)
    sampler = Sampler(session=session)
    
    # Submit multiple jobs in session
    for i in range(10):  # E.g., VQE iterations
        job = sampler.run([circuit])
        result = job.result()
        # Update circuit based on result
        
# Session auto-closes after context exit
""")

print("\n🎯 EXAM TIP: Sampler(session=session) NOT Sampler(backend=backend)!")

### Session Benefits

| Feature | Single Jobs | Session |
|---------|-------------|----------|
| Queue priority | Normal | Higher |
| Resource reservation | No | Yes |
| Latency between jobs | High | Low |
| Use case | One-off | Iterative (VQE) |

**EXAM TIP**: Sessions recommended for 3+ related jobs!

### ⚠️ EXAM TRAP: Session vs Backend Parameter

```python
# CORRECT - Use session parameter
with Session(backend=backend) as session:
    sampler = Sampler(session=session)  # ✓

# WRONG - Don't use backend in session
with Session(backend=backend) as session:
    sampler = Sampler(backend=backend)  # ❌
```

## Part 5: Batch Execution

### What is Batch?

**Batch** runs multiple **independent** circuits in parallel (unlike Session which is sequential).

```python
from qiskit_ibm_runtime import Batch, SamplerV2 as Sampler

with Batch(backend=backend) as batch:
    sampler = Sampler(mode=batch)
    
    # All circuits submitted together
    circuits = [qc1, qc2, qc3, qc4, qc5]
    job = sampler.run(circuits, shots=1024)
    
    # Results for all circuits
    results = job.result()
```

### Session vs Batch Decision

| Need | Use |
|------|-----|
| Sequential, dependent jobs (VQE) | **Session** |
| Parallel, independent circuits | **Batch** |
| Parameter sweeps | **Batch** |
| Iterative optimization | **Session** |

## Part 5: VQE Session Pattern

In [ ]:
# VQE pattern with Session
print("VQE with Session:")
print("""
with Session(backend=backend) as session:
    estimator = Estimator(session=session)
    
    # Optimization loop
    for iteration in range(max_iter):
        # Bind parameters
        bound_circuit = ansatz.assign_parameters(params)
        
        # Evaluate energy (blocks)
        job = estimator.run([bound_circuit], [hamiltonian])
        result = job.result()
        energy = result.values[0]
        
        # Update parameters
        params = optimizer.step(energy)
        
        if converged:
            break
""")

print("\n✓ Session keeps backend reserved")
print("✓ Minimal latency between iterations")
print("✓ Higher priority in queue")

## 📝 Practice Questions

### Question 1: Blocking Behavior

**Which method blocks until job completes?**

A) `job.status()`  
B) `job.queue_position()`  
C) `job.result()`  
D) `job.job_id()`

<details>
<summary>Answer</summary>

**C) `job.result()`**

```python
# Blocking (waits)
result = job.result()  # ✓ Blocks until DONE

# Non-blocking (immediate)
status = job.status()           # Returns immediately
position = job.queue_position() # Returns immediately
job_id = job.job_id()           # Returns immediately
```
</details>

---

### Question 2: Session Usage

**How to pass session to Sampler?**

A) `Sampler(backend=session)`  
B) `Sampler(session=session)`  
C) `Sampler(context=session)`  
D) `Sampler().set_session(session)`

<details>
<summary>Answer</summary>

**B) `Sampler(session=session)`**

```python
with Session(backend=backend) as session:
    sampler = Sampler(session=session)  # ✓ Correct
```

**CRITICAL**: Use `session` parameter, NOT `backend`!
</details>

---

### Question 3: When to Use Sessions

**When are Sessions recommended?**

A) Single circuit execution  
B) Iterative algorithms (VQE, QAOA)  
C) Only for simulators  
D) Never - deprecated

<details>
<summary>Answer</summary>

**B) Iterative algorithms (VQE, QAOA)**

Sessions best for:
- VQE optimization loops
- QAOA parameter sweeps
- Multiple related jobs (3+)

Benefits:
- Higher queue priority
- Reserved resources
- Lower latency between jobs
</details>

---

## ✅ Key Takeaways

### Core Concepts

1. **Job Lifecycle**
   - QUEUED → RUNNING → DONE
   - `job.result()` blocks
   - `job.status()` non-blocking

2. **Session Pattern**
   - `with Session(backend=backend) as session:`
   - `Sampler(session=session)` NOT `backend=backend`
   - Auto-closes after context exit

3. **Benefits**
   - Sessions: Higher priority, reserved resources
   - Recommended for 3+ related jobs
   - Critical for VQE/QAOA

### Critical Exam Facts

- ✅ `job.result()` BLOCKS until done
- ✅ `job.status()` returns immediately
- ✅ Session uses `session` parameter not `backend`
- ✅ Sessions recommended for iterative algorithms
- ✅ `job.job_id()` for retrieval
- ✅ Context manager auto-closes session

### Mnemonic

🧠 **"result() Waits, status() Checks, Session Groups!"**

**Next**: Options configuration!